In [9]:
import os
import tempfile
import subprocess
import logging
import argparse
from collections import OrderedDict
import requests

from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL

#import geopandas as gp
#from shapely.geometry import Point

import pandas as pd
from pandas.io.json import json_normalize
import pyproj as proj

In [10]:
# setup your projections
crs_wgs = proj.Proj(init='epsg:4326') # assuming you're using WGS84 geographic
crs_rd = proj.Proj(init='epsg:28992') # use a locally appropriate projected CRS


In [17]:
datadir = 'data/aanvalsplan_schoon/crow'
files = os.listdir(datadir)
files_xls = [f for f in files if f[-4:] == 'xlsx']
print(files_xls)

['data_export_afvalbakken_09-2017.xlsx', 'data_export_containers_07-2017.xlsx', 'data_export_afvalbakken_06-201.xlsx', 'data_export_afvalbakken_04-201-2.xlsx', 'data_export_afvalbakken_10-201.xlsx', 'data_export_afvalbakken_02-2017.xlsx', 'data_export_afvalbakken_08-2017.xlsx', '201706data.xlsx', 'data_export_containers_06-2017.xlsx', '201710data.xlsx', 'data_export_containers_08-2017.xlsx', 'data_export_containers_05-2017.xlsx', '201703data.xlsx', '201702data.xlsx', 'data_export_afvalbakken_05-201.xlsx', 'crowscores_2014_2016_feb2017.xlsx', 'data_export_afvalbakken_03-201.xlsx', 'data_export_containers_09-2017.xlsx', 'data_export_afvalbakken_07-2017.xlsx', 'data_export_containers_10-2017.xlsx', 'data_export_containers_04-2017.xlsx', '201704data.xlsx', '201705data.xlsx']


In [26]:
# Load all files into 1 big dataframe with lat lon as 4326
df = pd.DataFrame()
for f in files_xls:
    data = pd.read_excel(datadir + '/' + f)
    if ('Schouwronde') not in data.columns:
        data['Schouwronde'] = f
    #print(data.columns)
    # duplicate lat/lon
    if ('Latitude') in data.columns:
        data['lat'] = data['Latitude']
        data['lon'] = data['Longitude']
    # duplicate Breedtegraad
    if ('Breedtegraad') in data.columns:
        data['lat'] = data['Breedtegraad']
        data['lon'] = data['Lengtegraad']
    # convert RD bbox to lat lon
    if ('minx') in data.columns and ('lat') not in data.columns:
        data['RD-X'] = (data['minx'] + data['maxx']) / 2
        data['RD-Y'] = (data['miny'] + data['maxy']) / 2
        # convert RD N to WGS84 into Series
        latlon = data.apply(lambda row: proj.transform(crs_rd, crs_wgs, row['RD-X'],  row['RD-Y']),axis=1).apply(pd.Series)
        latlon.rename(columns={0: "lat", 1: "lon"},inplace=True)
        # Merge with dataFrame
        data = pd.concat([data,latlon], axis=1)
        #print(data)
    df = df.append(data)
    print("added " + f)




added data_export_afvalbakken_09-2017.xlsx
added data_export_containers_07-2017.xlsx
added data_export_afvalbakken_06-201.xlsx
added data_export_afvalbakken_04-201-2.xlsx
added data_export_afvalbakken_10-201.xlsx
added data_export_afvalbakken_02-2017.xlsx
added data_export_afvalbakken_08-2017.xlsx
added 201706data.xlsx
added data_export_containers_06-2017.xlsx
added 201710data.xlsx
added data_export_containers_08-2017.xlsx
added data_export_containers_05-2017.xlsx
added 201703data.xlsx
added 201702data.xlsx
added data_export_afvalbakken_05-201.xlsx
added crowscores_2014_2016_feb2017.xlsx
added data_export_afvalbakken_03-201.xlsx
added data_export_containers_09-2017.xlsx
added data_export_afvalbakken_07-2017.xlsx
added data_export_containers_10-2017.xlsx
added data_export_containers_04-2017.xlsx
added 201704data.xlsx
added 201705data.xlsx


In [25]:
df.head()
print(df.columns, df.dtypes)

df['Aanmaakdatum score']=df['Aanmaakdatum score'].apply(pd.to_datetime)
df.rename(columns={'Well ID (customer)': 'Well ID customer'},inplace=True)
print(df.columns, df.dtypes)
df.head()

Index(['Aanmaakdatum score', 'Adres', 'BU_CODE', 'BU_NAAM', 'Bestekspost',
       'Breedtegraad', 'Buurt', 'Containertype', 'Eigenaar', 'Fractie', 'ID',
       'ID-nummer', 'Inspecteur', 'Kleur', 'Latitude', 'Lengtegraad',
       'Longitude', 'Meetlocatienummer', 'Meetpunt', 'MsLink', 'RD-X', 'RD-Y',
       'Schaalniveau1', 'Schaalniveau2', 'Schaalniveau3', 'Schouwronde',
       'Score', 'Serienummer', 'Stadsdeel', 'Volgnummer inspectie',
       'Volgnummer score', 'Volume containertype', 'WK_CODE', 'WK_NAAM',
       'Well ID', 'Well ID (customer)', 'Wijk', 'X', 'Y', 'address', 'bc2015',
       'brtk2015', 'geb22', 'lat', 'lon', 'maxx', 'maxy', 'minx', 'miny',
       'name', 'verblijfin'],
      dtype='object') Aanmaakdatum score       object
Adres                    object
BU_CODE                  object
BU_NAAM                  object
Bestekspost              object
Breedtegraad            float64
Buurt                    object
Containertype            object
Eigenaar               

,Aanmaakdatum score,Adres,BU_CODE,BU_NAAM,Bestekspost,Breedtegraad,Buurt,Containertype,Eigenaar,Fractie,...,brtk2015,geb22,lat,lon,maxx,maxy,minx,miny,name,verblijfin
0,2017-08-15 09:28:55,NaN,NaN,NaN,Meubilair-afvalbak-vullingsgraad,52.387228,NaN,NaN,NaN,NaN,...,NaN,NaN,52.387228,4.876939,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-08-21 14:21:38,NaN,NaN,NaN,Meubilair-afvalbak-vullingsgraad,52.316841,NaN,NaN,NaN,NaN,...,NaN,NaN,52.316841,4.968766,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-08-21 12:11:21,NaN,NaN,NaN,Meubilair-afvalbak-vullingsgraad,52.325805,NaN,NaN,NaN,NaN,...,NaN,NaN,52.325805,4.962499,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-08-23 07:49:19,NaN,NaN,NaN,Meubilair-afvalbak-vullingsgraad,52.386985,NaN,NaN,NaN,NaN,...,NaN,NaN,52.386985,4.868285,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-08-14 06:37:15,NaN,NaN,NaN,Meubilair-afvalbak-vullingsgraad,52.387984,NaN,NaN,NaN,NaN,...,NaN,NaN,52.387984,4.822853,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Test projection on 1 row
#proj.transform(crs_rd, crs_wgs, data.loc[1,'RD-X'],data.loc[1,'RD-Y'])

(4.893024645571507, 52.37748696246846)

In [6]:
# add wkt text field
df['geometry'] = df.apply(lambda z: Point(z.lon, z.lat), axis=1)

In [11]:
#df['geometry'].head()

0    POINT (4.876939 52.387228)
1    POINT (4.968766 52.316841)
2    POINT (4.962499 52.325805)
3    POINT (4.868285 52.386985)
4    POINT (4.822853 52.387984)
Name: geometry, dtype: object

In [7]:
# convert to geodataframe
gdf = gp.GeoDataFrame(df)

In [8]:
gdf.crs = {'init': u'epsg:4326'}

In [9]:
points_gdf = gdf[gdf.is_valid == True]

In [10]:
def areaGeometry(url):
    data = requests.get(url)
    data = data.json()
    #print(data['features'][0])
    buurt_gdf = gp.GeoDataFrame(data['features'])
    #buurt_gdf.head()
    Properties = json_normalize(data['features'])
    #print(Properties)
    gdf = pd.concat([buurt_gdf.geometry, Properties.drop(['geometry.coordinates','geometry.type','type'],1)], axis=1)
    #gdf.head()
    return gdf

buurt = areaGeometry("https://map.data.amsterdam.nl/maps/gebieden?REQUEST=GetFeature&SERVICE=wfs&Version=1.1.0&SRSNAME=EPSG:4326&outputformat=geojson&typename=buurt")
stadsdeel = areaGeometry("https://map.data.amsterdam.nl/maps/gebieden?REQUEST=GetFeature&SERVICE=wfs&Version=1.1.0&SRSNAME=EPSG:4326&outputformat=geojson&typename=stadsdeel")
gebied = areaGeometry("https://map.data.amsterdam.nl/maps/gebieden?REQUEST=GetFeature&SERVICE=wfs&Version=1.1.0&SRSNAME=EPSG:4326&outputformat=geojson&typename=gebiedsgerichtwerken")

buurt.head()
stadsdeel.head()
gebied.head()



,geometry,properties.code,properties.display,properties.id,properties.naam,properties.type,properties.uri
0,"{'type': 'Polygon', 'coordinates': [[[4.878399...",DX11,Buitenveldert / Zuidas,DX11,Buitenveldert / Zuidas,gebieden/gebiedsgerichtwerken,https://api.data.amsterdam.nl/gebieden/gebieds...
1,"{'type': 'Polygon', 'coordinates': [[[4.900930...",DX02,Centrum-Oost,DX02,Centrum-Oost,gebieden/gebiedsgerichtwerken,https://api.data.amsterdam.nl/gebieden/gebieds...
2,"{'type': 'Polygon', 'coordinates': [[[4.758298...",DX06,Geuzenveld-Slotermeer-Sloterdijken,DX06,Geuzenveld-Slotermeer-Sloterdijken,gebieden/gebiedsgerichtwerken,https://api.data.amsterdam.nl/gebieden/gebieds...
3,"{'type': 'Polygon', 'coordinates': [[[4.865534...",DX10,Oud Zuid,DX10,Oud Zuid,gebieden/gebiedsgerichtwerken,https://api.data.amsterdam.nl/gebieden/gebieds...
4,"{'type': 'Polygon', 'coordinates': [[[4.910999...",DX14,Indische Buurt / Oostelijk Havengebied,DX14,Indische Buurt / Oostelijk Havengebied,gebieden/gebiedsgerichtwerken,https://api.data.amsterdam.nl/gebieden/gebieds...


In [11]:
points_in_stadsdeel = gp.sjoin(points_gdf, stadsdeel, how="inner", op='intersects')

ModuleNotFoundError: No module named 'rtree'